In [253]:
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import time
import matplotlib

# import skimage
from skimage import io, filters, exposure, measure, transform
from scipy.signal import find_peaks, savgol_filter

pd.set_option('mode.chained_assignment',None)

%matplotlib widget 
# %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})

SMALLER_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [75]:
testDataFolder = 'D://PMMH_ComputerData//Matlab Analysis//Data_Joseph//Raw//20.06.15_Deptho//depthoAnalysis_test'
# testDataFolder = 'C://Users//JosephVermeil//Desktop//test_AnalyseDepthos'
listDepthos = [f for f in os.listdir(testDataFolder) if (os.path.isfile(os.path.join(testDataFolder, f)) and f.endswith(".tif"))]
listDepthos

['Deptho100Xb3.tif',
 'Deptho100Xb4.tif',
 'Deptho100Xb5.tif',
 'Deptho100Xb6.tif']

In [76]:
fileName = listDepthos[0]
filePath = os.path.join(testDataFolder,fileName)
I = skimage.io.imread(filePath)

In [77]:
# if not I.ndim == 3:
#     print('Unexpected image format !')
# else:
#     I0 = I[0]

# fig, ax = plt.subplots(1,1)
# ax.imshow(I[2], cmap = 'gray')
# fig.tight_layout()
# fig.show()

# I.shape
# smallI = I[198:203]
# smallROI = I[:,100:400,100:500]
# # plt.imshow(smallROI, cmap = 'gray')
# threshold = skimage.filters.threshold_otsu(smallROI)
# threshold

# fig, ax = plt.subplots(1,1)
# ax.hist(smallROI.ravel(), bins=256, histtype='step', color='black')
# fig.show()

# np.zeros([nz, ny, nx])

In [78]:
nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
maxVal = np.zeros(nz)
for z in range(nz):
    maxVal[z] = np.max(I[z])

#
Z = np.array([i for i in range(nz)])
fig, ax = plt.subplots(1,1)
ax.plot(Z, maxVal)
fig.show()
#

zMax = np.argmax(maxVal)
IMax = I[zMax]

#
fig, ax = plt.subplots(1,1)
ax.imshow(IMax, cmap = 'gray')
fig.show()
#

threshold = skimage.filters.threshold_li(IMax)
binIMax = IMax > threshold

#
fig, ax = plt.subplots(1,1)
ax.imshow(binIMax, cmap = 'gray')
fig.show()
#

threshold = skimage.filters.threshold_otsu(IMax)
binIMax = IMax > threshold

#
fig, ax = plt.subplots(1,1)
ax.imshow(binIMax, cmap = 'gray')
fig.show()
#

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [356]:
class MultiBeadsDeptho:
   
    def __init__(self, I, scale):
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.__I = I
        self.__scale = scale
        self.__I_labeled = np.zeros([nz, ny, nx])
        self.__nz = nz
        self.__ny = ny
        self.__nx = nx
        self.__beadList = []
        self.__nB = 0
        
        
        
    def get_beadList(self):
        return(self.__beadList)
    
    def build_beadList(self):
        maxVal = np.zeros(self.__nz)
        for z in range(self.__nz):
            maxVal[z] = np.max(self.__I[z])
        z_max = np.argmax(maxVal)
        I_max = self.__I[z_max]
        threshold = skimage.filters.threshold_otsu(I_max)
        I_max_bin = (I_max > threshold)
        I_labeled, nB = measure.label(I_max_bin, return_num = True)
        self.__I_labeled, self.__nB = I_labeled, nB
        
#         for k in range(1,self.__nB+1):
#             ym0, xm0 = ndi.center_of_mass(I_max, I_labeled, k)
#             ym0, xm0 = int(np.floor(ym0)), int(np.floor(xm0))
#             I_labeled[ym0-1:ym0+2, xm0-1:xm0+2] = 2*nB
            
#         fig, ax = plt.subplots(1,1)
#         ax.imshow(I_labeled)
            
        for k in range(1,self.__nB+1):
#             print(k)
#             print(ndi.center_of_mass(I_max, I_labeled, k))
            ym0, xm0 = ndi.center_of_mass(I_max, I_labeled, k)
            self.__beadList.append(BeadDeptho(I, xm0, ym0, self.__scale))
            self.__beadList[-1].build_ROI()
            if self.__beadList[-1].validBead:
                self.__beadList[-1].build_zProfiles()
            
    
    
    @property
    def I(self):
        return(self.__I)
    
    @I.setter
    def I(self, I):
#         self.__I  =  I
        print('FORBIDEN: You cannot change the image after this object instantiation.')
    
    @property
    def nz(self):
        return(self.__nz)
    
    @property
    def ny(self):
        return(self.__ny)
    
    @property
    def nx(self):
        return(self.__nx)
    
    @property
    def nB(self):
        return(self.__nB)

    

In [373]:
class BeadDeptho:
    def __init__(self, I, xm0, ym0, scale):
        
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.scale = scale
        self.xm0 = xm0
        self.ym0 = ym0
        
        self.D = 0
        self.threshold = 0
        self.I_cleanROI = np.array([])
        self.XYm = np.zeros((2, self.nz), dtype = int)
        self.validBead = True
        
        self.z_max = 0
        self.validSlice = np.zeros(nz, dtype = bool)
        self.zFirst = 0
        self.zLast = nz
        self.validDepth = nz
        
        self.depthosDict = {}
        self.profileDict = {}
        self.ZfocusDict = {}
    
    
    def build_ROI(self):
        roughSize = np.floor(5*self.scale)
        roughSize += roughSize%2
        x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
        x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
        y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
        y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)

        if min([x1_roughROI,self.nx-x2_roughROI,y1_roughROI,self.ny-y2_roughROI]) < 0:
            self.validBead = False
            
        if self.validBead:
            I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            maxVal = np.zeros(self.nz)
            for z in range(self.nz):
                maxVal[z] = np.max(I_roughRoi[z])
            z_max = np.argmax(maxVal)
            self.z_max = z_max

#             print('Zmax = {:.0f}'.format(z_max))
#             figh, axh = plt.subplots(1,2, figsize = (8,4))
#             axh[0].imshow(I_roughRoi[z_max], cmap = 'gray')
#             axh[1].hist(I_roughRoi[z_max].ravel(), bins=256, histtype='step', color='black')
            
            

            counts, binEdges = np.histogram(I_roughRoi[z_max].ravel(), bins=256)

            peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
                               width=None, wlen=None, rel_height=0.5, plateau_size=None)

            if counts[peaks[0]] > 1200:
                self.D = 4.5
                roughSize = np.floor(2*self.D*self.scale)
                roughSize += roughSize%2
                x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
                x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
                y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
                y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)
                
            else:
                self.D = 2.7
                roughSize = np.floor(2*self.D*self.scale)
                roughSize += roughSize%2
                x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
                x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
                y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
                y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)
                
                
#             print('First peak height is {:.0f}'.format(counts[peaks[0]]))    
#             print('Detected D is {:.1f}'.format(self.D))
            
            if min([x1_roughROI,self.nx-x2_roughROI,y1_roughROI,self.ny-y2_roughROI]) < 0:
                self.validBead = False
                x1_roughROI, x2_roughROI = max(0, x1_roughROI), min(self.nx, x2_roughROI)
                y1_roughROI, y2_roughROI = max(0, y1_roughROI), min(self.nx, y2_roughROI) 
            
            I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            
    #         h_roughROI, w_roughROI = x2_roughROI-x1_roughROI, y2_roughROI-y1_roughROI
    #         print(h_roughROI//2, w_roughROI//2)
            
            if self.validBead:

                threshold = 0.8*skimage.filters.threshold_otsu(I_roughRoi)
                self.threshold = threshold
                I_roughRoi_binary = (I_roughRoi > threshold)
                I_roughRoi_binary_fh = ndi.binary_fill_holes(I_roughRoi_binary).astype(int)
                Ilab, nB_ROI = measure.label(I_roughRoi_binary_fh[z_max], return_num = True)
                if nB_ROI > 1:
                    self.validBead = False
                    
                if not self.validBead:
                    print('Not acceptable bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))
#                     fig, ax = plt.subplots(1,1, figsize = (4,4))
#                     ax.imshow(I_roughRoi[z_max], cmap = 'gray')
                
                if self.validBead:
                    area, circ = np.zeros(self.nz), np.zeros(self.nz)
                    for i in range(self.nz):
                        props = measure.regionprops_table(I_roughRoi_binary_fh[i], properties=('perimeter', 'area'))
                        if not np.sum(I_roughRoi_binary_fh[i]) == 0 and props['perimeter'][0] != 0:
                            area[i] = props['area'][0]
                            circ[i] = (4 * np.pi * props['area']) / (props['perimeter'] * props['perimeter'])[0]       
                    
                    validSlice = np.array((area > 50) & (circ > 0.8))
                    zFirst = np.argmax(validSlice)
                    zLast = self.nz - np.argmax(validSlice[::-1])

                    XYm = np.zeros((2, self.nz), dtype = int)
                    cleanSize = int(round(1*self.D*self.scale))
                    cleanSize += 1 + cleanSize%2
                    
                    I_cleanROI = np.zeros([self.nz, cleanSize, cleanSize])

                    try:
                        for i in range(zFirst, zLast):
                            Ym, Xm = ndi.center_of_mass(I_roughRoi[i], labels=I_roughRoi_binary_fh[i], index=1)
                            XYm[1, i], XYm[0, i] = Ym + y1_roughROI, Xm + x1_roughROI
                            roughCenter = int((roughSize+1)//2)
                            tform = transform.EuclideanTransform(rotation=0, \
                                                                 translation = (Xm-roughCenter, Ym-roughCenter))
                            I_tmp = transform.warp(I_roughRoi[i], tform, order = 1, preserve_range = True)

                            I_cleanROI[i] = np.copy(I_tmp[roughCenter-cleanSize//2:roughCenter+cleanSize//2+1,\
                                                          roughCenter-cleanSize//2:roughCenter+cleanSize//2+1])

                            # VISUALISE
                            if i == z_max:
#                                 print('Zfirst = {:.0f} and Zlast = {:.0f}'.format(zFirst, zLast))
                                print('Job done for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))

#                                 fig, ax = plt.subplots(1,4, figsize = (16,4))
#                                 ax[0].imshow(I_roughRoi[i], cmap = 'gray')
#                                 mid = I_roughRoi[i].shape[0]//2
#                                 ax[0].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
#                                 ax[0].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
#                                 I_max_binary = (I_roughRoi[i] > threshold)
#                                 y, x = ndi.center_of_mass(I_roughRoi[i], labels=I_max_binary, index=1)
#                                 ax[0].plot([x],[y], 'b+')

#                                 ax[1].imshow(I_tmp, cmap = 'gray')
#                                 mid = I_tmp.shape[0]//2
#                                 ax[1].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
#                                 ax[1].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
#                                 I_tmp_binary = (I_tmp > threshold)
#                                 y, x = ndi.center_of_mass(I_tmp, labels=I_tmp_binary, index=1)
#                                 ax[1].plot([x],[y], 'b+')

#                                 ax[2].imshow(I_cleanROI[i], cmap = 'gray')
#                                 mid = I_cleanROI[i].shape[0]//2
#                                 ax[2].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
#                                 ax[2].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
#                                 I_cleanRoi_binary = (I_cleanROI > threshold)
#                                 y, x = ndi.center_of_mass(I_cleanROI[i], labels=I_cleanRoi_binary[i], index=1)
#                                 ax[2].plot([x],[y], 'b+')

#                                 ax[3].hist(I_roughRoi[i].ravel(), bins=256, histtype='step', color='black')
                        
                        self.validSlice = validSlice
                        self.zFirst = zFirst
                        self.zLast = zLast
                        self.validDepth = zLast-zFirst
                        self.XYm = XYm
                        self.I_cleanROI = I_cleanROI.astype(np.uint16)

                    except:
                        print('BUG ! for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))

    #         ax.hist(I_roughRoi[z_max].ravel(), bins=256, histtype='step', color='black')
    #         print(XYm)
                          
    
    def build_zProfiles(self):
        side_ROI = self.I_cleanROI.shape[1]
        mid_ROI = side_ROI//2
        nbPixToAvg = 3 # Have to be an odd number
        deptho_v = np.zeros([self.nz, side_ROI], dtype = np.float64)
        deptho_h = np.zeros([self.nz, side_ROI], dtype = np.float64)
        
        for z in range(self.zFirst, self.zLast):
            templine = side_ROI
            deptho_v[z] = self.I_cleanROI[z,:,mid_ROI] * (1/nbPixToAvg)
#             print(self.I_cleanROI[z,:,mid_ROI])
#             print(1/nbPixToAvg)
#             print(self.deptho_v[z])
            deptho_h[z] = self.I_cleanROI[z,mid_ROI,:] * (1/nbPixToAvg)
    
            for i in range(1, 1 + nbPixToAvg//2):
#                 print(self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) )
                deptho_v[z] += self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) 
                deptho_v[z] += self.I_cleanROI[z,:,mid_ROI + i] * (1/nbPixToAvg)
                
                deptho_h[z] += self.I_cleanROI[z,mid_ROI - i,:] * (1/nbPixToAvg) 
                deptho_h[z] += self.I_cleanROI[z,mid_ROI + i,:] * (1/nbPixToAvg)
          
        deptho_v = deptho_v.astype(np.uint16)
        deptho_h = deptho_h.astype(np.uint16)
        
        self.depthosDict['deptho_v'] = deptho_v
        self.depthosDict['deptho_h'] = deptho_h
        
        
        # 3D caracterisation
        I_binary = np.zeros([self.I_cleanROI.shape[0], self.I_cleanROI.shape[1], self.I_cleanROI.shape[2]])
        I_binary[self.zFirst:self.zLast] = (self.I_cleanROI[self.zFirst:self.zLast] > self.threshold)
        Zm3D, Ym3D, Xm3D = ndi.center_of_mass(self.I_cleanROI, labels=I_binary, index=1)
        self.ZfocusDict['Zm3D'] = Zm3D
        
        
        # Raw profiles
        
        Z = np.array([z for z in range(self.I_cleanROI.shape[0])])
        intensity_tot = np.array([np.sum(self.I_cleanROI[z][I_binary[z].astype(bool)])/(1+np.sum(I_binary[z])) for z in range(self.I_cleanROI.shape[0])]).astype(np.float64)
        intensity_v = np.array([np.sum(deptho_v[z,:])/side_ROI for z in range(deptho_v.shape[0])]).astype(np.float64)
        intensity_h = np.array([np.sum(deptho_h[z,:])/side_ROI for z in range(deptho_h.shape[0])]).astype(np.float64)
        
        Zm_v, Zm_h, Zm_tot = np.argmax(intensity_v), np.argmax(intensity_h), np.argmax(intensity_tot)
        
        self.profileDict['intensity_v'] = intensity_v
        self.profileDict['intensity_h'] = intensity_h
        self.profileDict['intensity_tot'] = intensity_tot
        self.ZfocusDict['Zm_v'] = Zm_v
        self.ZfocusDict['Zm_h'] = Zm_h
        self.ZfocusDict['Zm_tot'] = Zm_tot
        

        # Smoothed profiles
        
        Z_hd = np.arange(0,self.I_cleanROI.shape[0],0.2)
        intensity_v_hd = np.interp(Z_hd, Z, intensity_v)
        intensity_h_hd = np.interp(Z_hd, Z, intensity_h)
        intensity_tot_hd = np.interp(Z_hd, Z, intensity_tot)
        intensity_v_smooth = savgol_filter(intensity_v_hd, 101, 7)
        intensity_h_smooth = savgol_filter(intensity_h_hd, 101, 7)
        intensity_tot_smooth = savgol_filter(intensity_tot_hd, 101, 7)
        
        Zm_v_hd, Zm_h_hd, Zm_tot_hd = Z_hd[np.argmax(intensity_v_smooth)], Z_hd[np.argmax(intensity_h_smooth)], Z_hd[np.argmax(intensity_tot_smooth)]
        
        self.profileDict['intensity_v_smooth'] = intensity_v_smooth 
        self.profileDict['intensity_h_smooth'] = intensity_h_smooth
        self.profileDict['intensity_tot_smooth'] = intensity_tot_smooth
        self.ZfocusDict['Zm_v_hd'] = Zm_v_hd 
        self.ZfocusDict['Zm_h_hd'] = Zm_h_hd
        self.ZfocusDict['Zm_tot_hd'] = Zm_tot_hd
        
        # VISUALISE
        
#         fig, ax = plt.subplots(1,3, figsize = (12, 4))
#         ax[0].plot(Z, intensity_v)
#         ax[1].plot(Z, intensity_h)
#         ax[2].plot(Z, (intensity_tot))
#         ax[0].plot([Zm_v, Zm_v], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v = {:.2f}'.format(Zm_v))
#         ax[1].plot([Zm_h, Zm_h], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h = {:.2f}'.format(Zm_h))
#         ax[2].plot([Zm_tot, Zm_tot], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot = {:.2f}'.format(Zm_tot))
#         ax[0].legend(loc = 'lower right')
#         ax[1].legend(loc = 'lower right')
#         ax[2].legend(loc = 'lower right')
        
        fig, ax = plt.subplots(1,3, figsize = (12, 4))
        ax[0].plot(Z, intensity_v, 'b-')
        ax[1].plot(Z, intensity_h, 'b-')
        ax[2].plot(Z, (intensity_tot), 'b-')
        ax[0].plot(Z_hd, intensity_v_smooth, 'k--')
        ax[1].plot(Z_hd, intensity_h_smooth, 'k--')
        ax[2].plot(Z_hd, intensity_tot_smooth, 'k--')
        ax[0].plot([Zm_v_hd, Zm_v_hd], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v_hd = {:.2f}'.format(Zm_v_hd))
        ax[1].plot([Zm_h_hd, Zm_h_hd], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h_hd = {:.2f}'.format(Zm_h_hd))
        ax[2].plot([Zm_tot_hd, Zm_tot_hd], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot_hd = {:.2f}'.format(Zm_tot_hd))
        ax[0].legend(loc = 'lower right')
        ax[1].legend(loc = 'lower right')
        ax[2].legend(loc = 'lower right')
        
        print('Zm_v = {:.2f}, Zm_h = {:.2f}, Zm_tot = {:.2f}'\
              .format(Zm_v, Zm_h, Zm_tot))
        print('Zm_v_hd = {:.2f}, Zm_h_hd = {:.2f}, Zm_tot_hd = {:.2f}'\
              .format(Zm_v_hd, Zm_h_hd, Zm_tot_hd))
        print('Xm3D = {:.2f}, Ym3D = {:.2f}, Zm3D = {:.2f}'\
              .format(Xm3D, Ym3D, Zm3D))
        print('\n')
        
        return(0)

In [374]:
mbd = MultiBeadsDeptho(I, 15.8)
mbd.nB

0

In [375]:
mbd.build_beadList()
mbd.nB
listBeads = mbd.get_beadList()

Job done for the bead in x0, y0 = 182.24, 143.52


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Zm_v = 169.00, Zm_h = 169.00, Zm_tot = 165.00
Zm_v_hd = 169.60, Zm_h_hd = 167.00, Zm_tot_hd = 165.40
Xm3D = 36.01, Ym3D = 36.00, Zm3D = 193.96


Job done for the bead in x0, y0 = 353.88, 147.96


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Zm_v = 173.00, Zm_h = 170.00, Zm_tot = 174.00
Zm_v_hd = 172.00, Zm_h_hd = 168.60, Zm_tot_hd = 171.60
Xm3D = 36.01, Ym3D = 36.00, Zm3D = 187.17


Job done for the bead in x0, y0 = 1601.62, 149.54


...

Zm_v = 176.00, Zm_h = 176.00, Zm_tot = 176.00
Zm_v_hd = 174.40, Zm_h_hd = 176.20, Zm_tot_hd = 176.80
Xm3D = 36.00, Ym3D = 36.00, Zm3D = 196.23


Job done for the bead in x0, y0 = 278.68, 323.96


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Zm_v = 173.00, Zm_h = 176.00, Zm_tot = 175.00
Zm_v_hd = 175.20, Zm_h_hd = 176.80, Zm_tot_hd = 176.40
Xm3D = 36.01, Ym3D = 36.00, Zm3D = 201.85


Not acceptable bead in x0, y0 = 1814.52, 384.78
Not acceptable bead in x0, y0 = 1811.72, 455.93
Job done for the bead in x0, y0 = 522.14, 493.35


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Zm_v = 184.00, Zm_h = 182.00, Zm_tot = 184.00
Zm_v_hd = 182.60, Zm_h_hd = 183.60, Zm_tot_hd = 182.20
Xm3D = 36.00, Ym3D = 36.00, Zm3D = 207.96




In [372]:
# b0 = listBeads[2]
# ni = 5
# fig, ax = plt.subplots(ni,1, figsize = (6, 5*ni))
# for i in range(0,ni):
#     z = b0.zFirst+i
#     ax[i].imshow(mbd.I[z], cmap='gray')
#     ax[i].plot(b0.XYm[0,z],b0.XYm[1,z], 'b+')

np.arange(0,1,0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [245]:
for bD in listBeads:
    if bD.validBead:
        fig, ax = plt.subplots(1,2, figsize = (8, 5))
        ax[0].imshow(bD.deptho_h, cmap='gray')
        ax[1].imshow(bD.deptho_v, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [222]:
bD = listBeads[0]
bD.intensity_v

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
       363274, 366605, 369248, 372837, 377347, 381091, 383494, 388589,
      

In [223]:
bD.intensity_h

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
       360855, 363932, 367701, 371395, 375791, 380219, 383286, 386857,
      

In [225]:
bD.intensity_v - bD.intensity_h

array([         0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,          0,        

In [139]:
plt.close('all')

In [ ]:
mbd.I

In [195]:
A = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
A

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [197]:
np.sum(A[0])

6

In [288]:
O = np.zeros([3, 2, 2])
O[1,:,:] = A[:2,:2]
O[2,:,:] = A[1:,1:]
O

array([[[0., 0.],
        [0., 0.]],

       [[1., 2.],
        [4., 5.]],

       [[5., 6.],
        [8., 9.]]])

In [292]:
b = np.array([1, 0], dtype = bool)
O[0][b]

array([[0., 0.]])

In [157]:

O

array([[[0., 0.],
        [0., 0.]],

       [[1., 2.],
        [4., 5.]],

       [[0., 0.],
        [0., 0.]]])

In [160]:
A[0,0] = 0
A

array([[0, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [159]:
O

array([[[0., 0.],
        [0., 0.]],

       [[1., 2.],
        [4., 5.]],

       [[0., 0.],
        [0., 0.]]])